In [1]:

import os, shutil
from glob import glob
import numpy as np
import pandas as pd
from natsort import natsorted
import seaborn as sns
from tqdm import tqdm, trange
import matplotlib.pyplot as plt


In [2]:

in_dir = "out01_tms_induced_respose/"
in_dir2 = "data_subject_info/"

out_dir = "out02_multiple_regression_common_mask/"
in_file_name = "HRF_tms_induced_response_common_mask.csv"

if not os.path.exists(out_dir):
    os.mkdir(out_dir)
    
complete_df = pd.read_csv(in_dir + in_file_name)

subject_info = pd.read_csv(in_dir2 + "age_gender_edu.txt", sep = ',')
intensity_suds = pd.read_csv(in_dir2 + "intensity_suds.csv", sep = ',')
scalp_dist = pd.read_csv(in_dir2 + "dist_to_scalp.csv", sep = ',')
print(list(scalp_dist['site'].drop_duplicates()))


['L-FP', 'R-FP', 'L-aMFG', 'R-aMFG', 'L-pMFG', 'R-pMFG', 'R-IFJ', 'R-FEF', 'R-M1', 'R-preSMA', 'R-IPL']


In [3]:
# scalp_dist['site'] = scalp_dist['site'].str.replace('-', '_')
# df['range'].str.replace(',','-')

tms_sites = ["L_Fp","R_Fp","L_aMFG","R_aMFG","L_pMFG","R_pMFG","R_IFJ","R_FEF","R_M1","R_preSMA","R_IPL"]
scalp_dist['site'].replace(list(scalp_dist['site'].drop_duplicates()), tms_sites, inplace = True)

# reformat columns in intensity_suds:
intensity_suds = pd.melt(intensity_suds, id_vars=['idall', 'MT', 'intensity'], value_vars = tms_sites,
                         var_name = 'suds_site', value_name = 'suds')

data = subject_info.merge(complete_df, left_on = 'cc_post_intake_id', right_on = 'subject')
data = data.merge(intensity_suds, how = 'left', 
                  left_on = ['cc_post_intake_id', 'site'], right_on = ['idall', 'suds_site'])

data = data.merge(scalp_dist, how = 'left', 
                  left_on = ['subject', 'site'], right_on = ['subject', 'site'])

data.drop(labels = ['inputfile', 'idall', 'cc_post_intake_id', 'suds_site'], axis = 1, inplace = True)
# data.replace({'gender': {1: "male", 2: "female"}}, inplace = True)
# data.replace({'gender': {'male': 1, 'female': 2}}, inplace = True)

data


,gender,age,yrs_of_edu,subject,site,group,tms_site_response_6mm,tms_site_response_10mm,tms_site_response_14mm,tms_site_response_14-10mm,tms_site_response_10-6mm,MT,intensity,suds,scalp_dist
0,2,45,18,1001,L_Fp,NTHC,0.183782,0.223570,0.179095,0.151442,0.234871,62.0,74.0,10.0,14.749354
1,2,45,18,1001,L_pMFG,NTHC,-0.444990,-0.412696,-0.310918,-0.256227,-0.406768,62.0,74.0,5.0,13.038824
2,2,45,18,1001,R_FEF,NTHC,-0.311842,-0.273876,-0.276212,-0.277628,-0.262818,62.0,74.0,2.0,17.079627
3,2,45,18,1001,R_Fp,NTHC,0.247076,0.241324,0.243161,0.244355,0.239672,62.0,74.0,7.0,16.614293
4,2,45,18,1001,R_M1,NTHC,-0.916150,-1.146695,-1.232267,-1.289315,-1.221521,62.0,74.0,1.0,15.276347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,1,22,14,2108,R_IPL,TEHC,0.644515,1.131969,1.284253,1.393884,1.307835,NaN,100.0,5.0,14.361543
788,1,22,14,2108,R_M1,TEHC,0.210817,0.194829,0.193421,0.192483,0.189640,NaN,100.0,15.0,18.975313
789,1,22,14,2108,R_aMFG,TEHC,0.058481,-0.014030,-0.102940,-0.160327,-0.035552,NaN,100.0,20.0,13.567683
790,1,22,14,2108,R_pMFG,TEHC,0.590151,0.451812,0.335860,0.267050,0.416303,NaN,100.0,20.0,14.366030


In [4]:

## correlation matrix:
d = data.loc[:, ['gender', 'age', 'yrs_of_edu', 'MT', 'intensity', 'suds', 'scalp_dist']]
print(d.count())
d.corr()


gender        792
age           792
yrs_of_edu    792
MT            581
intensity     792
suds          771
scalp_dist    689
dtype: int64


,gender,age,yrs_of_edu,MT,intensity,suds,scalp_dist
gender,1.000000,0.100431,0.030555,0.026028,-0.005011,-0.027004,0.086767
age,0.100431,1.000000,0.430420,-0.219282,-0.164505,0.079981,0.078526
yrs_of_edu,0.030555,0.430420,1.000000,-0.085059,-0.087906,-0.001968,0.044035
MT,0.026028,-0.219282,-0.085059,1.000000,0.999374,0.011761,0.134476
intensity,-0.005011,-0.164505,-0.087906,0.999374,1.000000,0.114597,0.199412
suds,-0.027004,0.079981,-0.001968,0.011761,0.114597,1.000000,-0.125881
scalp_dist,0.086767,0.078526,0.044035,0.134476,0.199412,-0.125881,1.000000


In [5]:
import statsmodels
statsmodels.__version__

'0.13.5'

In [7]:
import statsmodels.api as sm

# backward selection

test_variables = ['tms_site_response_6mm',
           'tms_site_response_10mm', 'tms_site_response_14mm', 
           'tms_site_response_14-10mm', 'tms_site_response_10-6mm']

roi_list = data.site.unique()
feature = ['gender', 'age', 'yrs_of_edu', 'intensity', 'suds', 'scalp_dist']

row_index = pd.MultiIndex.from_tuples([(i , j) for i in test_variables for j in feature])
result = pd.DataFrame(index = row_index, columns = tms_sites)

for var in test_variables:    
    for roi in roi_list:
#         print(var)
#         print(roi)
        
        X = data.loc[(data['site'] == roi) & (~data[var].isna()), feature]
        y = data.loc[(data['site'] == roi) & (~data[var].isna()), [var]]

        if len(y) < 20: continue
            
        # remove nan in X:
        row_nan = X.isna().any(axis=1)
        X = sm.add_constant(X.loc[~row_nan,:]) # adding a constant

        model = sm.OLS(y.loc[~row_nan], X).fit()
        predictions = model.predict(X) 

#         print(model.summary())
        for i, f in enumerate(feature):
            result.loc[(var, f), roi] = model.pvalues[i]        


In [8]:

def format_table(report):
    report2 = report.astype(str).apply(lambda x : x.str[:5])
    
    report2[report.le(0.05)] = \
    report2[report.le(0.05)].apply(lambda x : x.str[:5]).add('*')

    report2[report.le(0.01)] = \
    report2[report.le(0.01)].apply(lambda x : x.str[:5]).add('**')
        
    return report2

result2 = format_table(result)
result2.to_csv(out_dir + "multiple_regression_pvalues_all.csv")

for var in test_variables:
    result2.loc[(var, feature),:].to_csv(out_dir + "multiple_regression_pvalues_" + var + ".csv")

result2


L_Fp   R_Fp  L_aMFG R_aMFG L_pMFG  \
tms_site_response_6mm     gender       0.216  0.698   0.247  0.075  0.865   
                          age          0.459  0.875  0.043*  0.740  0.533   
                          yrs_of_edu   0.200  0.693  0.036*  0.554  0.756   
                          intensity    0.294  0.472   0.422  0.227  0.333   
                          suds         0.839  0.535   0.052  0.574  0.978   
                          scalp_dist  0.023*  0.969   0.602  0.828  0.199   
tms_site_response_10mm    gender       0.104  0.882   0.304  0.067  0.951   
                          age          0.302  0.841  0.043*  0.482  0.611   
                          yrs_of_edu   0.212  0.594  0.048*  0.629  0.718   
                          intensity    0.315  0.480   0.414  0.271  0.462   
                          suds         0.482  0.639   0.080  0.686  0.896   
                          scalp_dist   0.081  0.896   0.717  0.569  0.261   
tms_site_response_14mm    gender       0.059  0.936   0.340  0.137  0.856   
                          age          0.294  0.873  0.030*  0.459  0.879   
                          yrs_of_edu   0.312  0.601   0.068  0.663  0.631   
                          intensity    0.279  0.609   0.471  0.345  0.558   
                          suds         0.296  0.757   0.115  0.971  0.764   
                          scalp_dist   0.170  0.779   0.934  0.531  0.375   
tms_site_response_14-10mm gender       0.055  0.978   0.379  0.248  0.740   
                          age          0.323  0.899  0.029*  0.459  0.936   
                          yrs_of_edu   0.431  0.610   0.093  0.702  0.586   
                          intensity    0.291  0.720   0.521  0.432  0.635   
                          suds         0.234  0.852   0.154  0.800  0.688   
                          scalp_dist   0.300  0.699   0.939  0.518  0.475   
tms_site_response_10-6mm  gender       0.087  0.943   0.322  0.069  0.970   
                          age          0.272  0.831  0.046*  0.413  0.631   
                          yrs_of_edu   0.229  0.567   0.053  0.663  0.711   
                          intensity    0.335  0.485   0.418  0.296  0.496   
                          suds         0.397  0.677   0.091  0.732  0.879   
                          scalp_dist   0.127  0.873   0.744  0.496  0.279   

                                     R_pMFG   R_IFJ  R_FEF   R_M1 R_preSMA  \
tms_site_response_6mm     gender      0.647   0.094  0.558  0.153    0.695   
                          age         0.541   0.585  0.788  0.887    0.214   
                          yrs_of_edu  0.968   0.792  0.698  0.535    0.142   
                          intensity   0.834   0.825  0.174  0.054    0.294   
                          suds        0.895  0.043*  0.463  0.901    0.468   
                          scalp_dist  0.597   0.267  0.815  0.955    0.902   
tms_site_response_10mm    gender      0.771   0.073  0.478  0.344    0.575   
                          age         0.299   0.668  0.628  0.791    0.166   
                          yrs_of_edu  0.518   0.876  0.907  0.612    0.155   
                          intensity   0.684   0.802  0.162  0.099    0.395   
                          suds        0.956  0.032*  0.428  0.889    0.319   
                          scalp_dist  0.840   0.190  0.741  0.956    0.908   
tms_site_response_14mm    gender      0.850   0.059  0.446  0.608    0.631   
                          age         0.172   0.711  0.659  0.791    0.128   
                          yrs_of_edu  0.478   0.908  0.865  0.502    0.154   
                          intensity   0.648   0.692  0.158  0.140    0.461   
                          suds        0.953  0.036*  0.490  0.656    0.315   
                          scalp_dist  0.929   0.165  0.777  0.803    0.927   
tms_site_response_14-10mm gender      0.907   0.057  0.438  0.852    0.669   
                          age         0.122   0.750  0.690  0.794    0.111   
              